In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from mlxtend.preprocessing import TransactionEncoder
import random
#from mlxtend.frequent_patterns import apriori
from apyori import apriori 
from itertools import combinations, groupby
from collections import Counter

In [2]:
from itertools import combinations, groupby
from collections import Counter

In [3]:
#load data
# based off https://www.datatheque.com/posts/association-analysis/

client = boto3.client('s3')
obj = client.get_object(Bucket='manifolddata', Key='week1.csv')
df = pd.read_csv(BytesIO(obj['Body'].read()))


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df=df.iloc[:,[0,1,3,4,5,6,7,8]]
df.columns=['Date', 'Duration', 'Src_IP', 'Src_pt', 'Dst_IP', 'Dst_pt','Packets', 'Bytes']
#add an date column that is rounded to nearest hour, so we can use this as a timestep to see how frequently IP pairs occur in each timestep
df['Date']=pd.to_datetime(df['Date'], format="%Y-%m-%d %H:%M:%S.%f", errors = 'coerce')
df['date_hr']=pd.Series(df['Date']).dt.round("H")

In [5]:
#create a pair column, which is a touple of the src and dst IP, sorted. 
#It does not matter which call came first, we simply want to know which pair occurs most frequently.

df['pairs']=list(zip(df.Src_IP, df.Dst_IP))
df['pairs']=df['pairs'].apply(sorted)
df['pairs2']=tuple(df['pairs'])

In [6]:
data=df[['Date', 'Src_IP', 'Dst_IP']]
#melt 
data2=data
data2=pd.melt(data2, id_vars=['Date'])

In [7]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
# Convert from DataFrame to a Series, with order_id as index and item_id as value
#necessary for assocation mining
data2=data2[['Date', 'value']]
data2.columns=['Date', 'IP']
data_series = data2.set_index('Date')['IP'].rename('IP')

type(data_series)

pandas.core.series.Series

In [8]:
data_series.loc['2017-08-02 00:00:00.419']# yes we get two results, 1 for source and 1 for destination

Date
2017-08-02 00:00:00.419    192.168.210.55
2017-08-02 00:00:00.419    192.168.100.11
Name: IP, dtype: object

In [9]:
#Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]   

In [10]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [11]:
#run the model

rules = association_rules(data_series, 0.01)  #0.01 is minimum support or that the IP address must appear in the requests 1% of time

Starting order_item:               16371984
Items with support >= 0.01:            1266
Remaining order_item:              15563374
Remaining orders with 2+ items:     6156085
Remaining order_item:              15009362


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Item pairs:                           60330
Item pairs with support >= 0.01:         82



In [12]:
#add back in the pairs column so we can match it up with our dataframe

rules['pairs']=list(zip(rules.item_A, rules.item_B))
rules['pairs']=rules['pairs'].apply(sorted)

How many rules did we create?

In [77]:
len(rules)

82

82 rules is not a lot, considering we have 44,000 unique pairs of IP addresses

In [13]:
rules.head()

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift,pairs
67,11761_123,11761_123,761,0.012362,2229,0.036208,2229,0.036208,0.341409,0.341409,9.429076,"[11761_123, 11761_123]"
25,10222_56,10222_56,644,0.010461,2645,0.042966,2645,0.042966,0.243478,0.243478,5.666816,"[10222_56, 10222_56]"
64,10085_154,10085_154,752,0.012216,4279,0.069508,4279,0.069508,0.175742,0.175742,2.528354,"[10085_154, 10085_154]"
27,10228_208,10228_208,792,0.012865,4512,0.073293,4512,0.073293,0.175532,0.175532,2.394923,"[10228_208, 10228_208]"
81,10005_251,10005_251,754,0.012248,5971,0.096993,5971,0.096993,0.126277,0.126277,1.301913,"[10005_251, 10005_251]"


## Create Scheduler

Assume we have enough servers to host each app (IP address), and each server can hold 20 apps. How many servers do we need?

In [14]:
apps_server=20
num_server=int(df['Dst_IP'].nunique()/apps_server)

In [15]:
int(num_server)

467

We know we can only fit 20 apps on each server, so in terms of the rules dataframe this is 10 rows (because each row contains 2 apps).
Create a sequence that fills in the server number on the rules dataframe, then merge these servers into the entire dataset.

In [16]:
#how many servers will we use for our rules
import math
pairs_server=apps_server/2

#how many servers do we need for our rules, which are in pairs?
servers_rule=math.ceil(len(rules)/pairs_server)

In [17]:
servers_rule_list=list(range(0,servers_rule+1))
servers_rule_list=np.repeat(servers_rule_list,pairs_server)

#remove the extra items
servers_rule_list=servers_rule_list[0:len(rules)]

In [18]:
rules['server']=servers_rule_list

In [28]:
rules.head()

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift,pairs,server
67,11761_123,11761_123,761,0.012362,2229,0.036208,2229,0.036208,0.341409,0.341409,9.429076,"[11761_123, 11761_123]",0
25,10222_56,10222_56,644,0.010461,2645,0.042966,2645,0.042966,0.243478,0.243478,5.666816,"[10222_56, 10222_56]",0
64,10085_154,10085_154,752,0.012216,4279,0.069508,4279,0.069508,0.175742,0.175742,2.528354,"[10085_154, 10085_154]",0
27,10228_208,10228_208,792,0.012865,4512,0.073293,4512,0.073293,0.175532,0.175532,2.394923,"[10228_208, 10228_208]",0
81,10005_251,10005_251,754,0.012248,5971,0.096993,5971,0.096993,0.126277,0.126277,1.301913,"[10005_251, 10005_251]",0


In [19]:
rules_A=rules[['item_A', 'server']]
rules_B=rules[['item_B', 'server']]
rules_A.columns=['item', 'server']
rules_B.columns=['item', 'server']

In [20]:
rules_AB=pd.concat([rules_A, rules_B])

In [21]:
rules_AB.head()

,item,server
67,11761_123,0
25,10222_56,0
64,10085_154,0
27,10228_208,0
81,10005_251,0


### merge in the servers from the rules to the  main dataframe

In [22]:
df2=df


,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2
0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)"
1,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)"
2,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)"
3,2017-08-02 00:00:02.859,0.000,10000_34,443,192.168.210.54,59628.0,1,100,2017-08-02,"[10000_34, 192.168.210.54]","(10000_34, 192.168.210.54)"
4,2017-08-02 00:00:02.594,0.000,192.168.100.11,445,192.168.220.47,55101.0,1,108,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)"


In [23]:
df2=df2.merge(rules_AB, left_on='Src_IP', right_on='item', how='left') #creates a column that shows the server used for Src_IP


In [24]:
df2=df2.rename(columns = {'server':'server_Src'})

,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2,item,server_Src
0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",192.168.210.55,7.0
1,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",192.168.210.55,7.0
2,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",NaN,NaN
3,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",192.168.220.47,7.0
4,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",192.168.220.47,7.0


In [25]:
df2=df2.merge(rules_AB, left_on='Dst_IP', right_on='item', how='left') #creates a column that shows the server used for Src_IP
 #creates a column that shows the server used for Dst_IP, or 'itemB' in the rules

In [26]:
df2=df2.rename(columns = {'server':'server_Dst'})


,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2,item_x,server_Src,item_y,server_Dst
0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",192.168.210.55,7.0,NaN,NaN
1,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",192.168.210.55,7.0,NaN,NaN
2,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",NaN,NaN,192.168.210.55,7.0
3,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",NaN,NaN,192.168.210.55,7.0
4,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",192.168.220.47,7.0,NaN,NaN


In [32]:
df2=df2.drop(['item_y', 'item_x'], axis=1)

create new column with the predicted durationo. if servers are the same for the pair, set duration=0

In [71]:
df2['duration_pred']=df2['Duration']
df2['duration_pred'][df2['server_Src']==df2['server_Dst']]=0

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [72]:
#want to check that we do indeed have matching servers for the rules we created
df2[df2['server_Src']==df2['server_Dst']]

,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2,server_Src,server_Dst,duration_pred
30485,2017-08-02 04:00:44.530,0.000,192.168.210.2,67,192.168.210.55,68.0,1,375,2017-08-02 04:00:00,"[192.168.210.2, 192.168.210.55]","(192.168.210.2, 192.168.210.55)",7.0,7.0,0.0
30486,2017-08-02 04:00:44.530,0.000,192.168.210.2,67,192.168.210.55,68.0,1,375,2017-08-02 04:00:00,"[192.168.210.2, 192.168.210.55]","(192.168.210.2, 192.168.210.55)",7.0,7.0,0.0
30487,2017-08-02 04:00:44.530,0.000,192.168.210.2,67,192.168.210.55,68.0,1,375,2017-08-02 04:00:00,"[192.168.210.2, 192.168.210.55]","(192.168.210.2, 192.168.210.55)",7.0,7.0,0.0
30488,2017-08-02 04:00:44.530,0.000,192.168.210.2,67,192.168.210.55,68.0,1,375,2017-08-02 04:00:00,"[192.168.210.2, 192.168.210.55]","(192.168.210.2, 192.168.210.55)",7.0,7.0,0.0
30489,2017-08-02 04:00:44.525,0.000,192.168.210.55,68,192.168.210.2,67.0,1,342,2017-08-02 04:00:00,"[192.168.210.2, 192.168.210.55]","(192.168.210.2, 192.168.210.55)",7.0,7.0,0.0
30490,2017-08-02 04:00:44.525,0.000,192.168.210.55,68,192.168.210.2,67.0,1,342,2017-08-02 04:00:00,"[192.168.210.2, 192.168.210.55]","(192.168.210.2, 192.168.210.55)",7.0,7.0,0.0
30491,2017-08-02 04:00:44.525,0.000,192.168.210.55,68,192.168.210.2,67.0,1,342,2017-08-02 04:00:00,"[192.168.210.2, 192.168.210.55]","(192.168.210.2, 192.168.210.55)",7.0,7.0,0.0
30492,2017-08-02 04:00:44.525,0.000,192.168.210.55,68,192.168.210.2,67.0,1,342,2017-08-02 04:00:00,"[192.168.210.2, 192.168.210.55]","(192.168.210.2, 192.168.210.55)",7.0,7.0,0.0
45864,2017-08-02 06:03:19.583,0.143,10042_29,80,192.168.220.46,42768.0,3,1002,2017-08-02 06:00:00,"[10042_29, 192.168.220.46]","(10042_29, 192.168.220.46)",6.0,6.0,0.0
45865,2017-08-02 06:03:19.583,0.143,10042_29,80,192.168.220.46,42768.0,3,1002,2017-08-02 06:00:00,"[10042_29, 192.168.220.46]","(10042_29, 192.168.220.46)",6.0,6.0,0.0


Sum up the originial duration times for latency, and compare that with our predicted latency times

In [73]:
df2['Duration'].sum()

7001653.546000004

In [74]:
df2['duration_pred'].sum()

6887416.561999986

% Difference:

In [76]:
(df2['Duration'].sum()- df2['duration_pred'].sum())/ df2['Duration'].sum()

0.016315715030670242

a 1% improvement

Now, fill in the rest of the servers based on random chance.

In [44]:
#We have these many servers to pick from:
int(num_server)

467

In [47]:
random.randint(0,467)

123

In [52]:
for item, frame in df2_check['server_Src'].iteritems():
    if pd.isnull(frame):
        df2_check['server_Src']= random.randint(0,467)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [57]:
for item in df2_check['server_Src']:
    if pd.isnull(item):
        item='yes'

In [ ]:
for i in df.index:
    if pd.isnull(df2_check['server_Src']):
        df.at[i, 'ifor'] = x
    else:
        df.at[i, 'ifor'] = y

In [ ]:
for i, row in df.iterrows():
  ifor_val = something
  if <condition>:
    ifor_val = something_else
  df.set_value(i,'ifor',ifor_val)

In [60]:
for i, row in df2_check.iterrows():
  if server_Src_val:
    server_Src_val = 'something_else'
  df2_check.set_value(i,'server_Src',server_Src_val)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [58]:
df2_check

,Date,Duration,Src_IP,Src_pt,Dst_IP,Dst_pt,Packets,Bytes,date_hr,pairs,pairs2,server_Src,server_Dst
0,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",7.0,NaN
1,2017-08-02 00:00:00.419,0.003,192.168.210.55,44870,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",7.0,NaN
2,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",NaN,7.0
3,2017-08-02 00:00:00.421,0.000,192.168.100.11,445,192.168.210.55,44870.0,1,108,2017-08-02,"[192.168.100.11, 192.168.210.55]","(192.168.100.11, 192.168.210.55)",NaN,7.0
4,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",7.0,NaN
5,2017-08-02 00:00:02.593,0.004,192.168.220.47,55101,192.168.100.11,445.0,2,174,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",7.0,NaN
6,2017-08-02 00:00:02.859,0.000,10000_34,443,192.168.210.54,59628.0,1,100,2017-08-02,"[10000_34, 192.168.210.54]","(10000_34, 192.168.210.54)",NaN,6.0
7,2017-08-02 00:00:02.859,0.000,10000_34,443,192.168.210.54,59628.0,1,100,2017-08-02,"[10000_34, 192.168.210.54]","(10000_34, 192.168.210.54)",NaN,6.0
8,2017-08-02 00:00:02.594,0.000,192.168.100.11,445,192.168.220.47,55101.0,1,108,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",NaN,7.0
9,2017-08-02 00:00:02.594,0.000,192.168.100.11,445,192.168.220.47,55101.0,1,108,2017-08-02,"[192.168.100.11, 192.168.220.47]","(192.168.100.11, 192.168.220.47)",NaN,7.0
